In [2]:
import os
from six.moves import cPickle as pickle
import platform
import numpy as np
 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))
def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000,32,32,3)
        Y = np.array(Y)
        return X, Y
def load_CIFAR10(ROOT):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, '/Users/taraapple/Desktop/cifar-10-batches-py/data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte
 
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    cifar10_dir = '/Users/taraapple/Desktop/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    x_train = X_train.astype('float32')
    x_val = X_val.astype('float32')
    x_test = X_test.astype('float32')
    x_train /= 255
    x_val /= 255
    x_test /= 255
    return x_train, y_train, x_val, y_val, x_test, y_test
 

x_train, y_train, x_val, y_val, x_test, y_test = get_CIFAR10_data()
 
print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense,Activation,Dropout,Flatten
from keras.utils import np_utils
from keras import *
import tensorflow as tf
from keras.optimizers import SGD
import os
from six.moves import cPickle as pickle
import platform
import numpy as np

In [5]:
batch_size = 128
nb_classes = 10
nb_epoch = 12
print("{1} train samples, {2} channel{0}, {3}x{4}".format("" if x_train.shape[1] == 1 else "s", *x_train.shape))
print("{1}  test samples, {2} channel{0}, {3}x{4}".format("" if x_test.shape[1] == 1 else "s", *x_test.shape))
print("{1}  val samples, {2} channel{0}, {3}x{4}".format("" if x_val.shape[1] == 1 else "s", *x_val.shape))
_, img_channels, img_rows, img_cols = x_train.shape
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
y_val = np_utils.to_categorical(y_val, nb_classes)
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=12, verbose=1, validation_data=(x_val, y_val))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

49000 train samples, 32 channels, 32x3
10000  test samples, 32 channels, 32x3
1000  val samples, 32 channels, 32x3
Epoch 1/12
383/383 [==============================] - 72s 189ms/step - loss: 1.9547 - accuracy: 0.2794 - val_loss: 1.5611 - val_accuracy: 0.4400
Epoch 2/12
383/383 [==============================] - 78s 203ms/step - loss: 1.5844 - accuracy: 0.4250 - val_loss: 1.3459 - val_accuracy: 0.5100
Epoch 3/12
383/383 [==============================] - 79s 206ms/step - loss: 1.4432 - accuracy: 0.4802 - val_loss: 1.2775 - val_accuracy: 0.5360
Epoch 4/12
383/383 [==============================] - 77s 202ms/step - loss: 1.3510 - accuracy: 0.5173 - val_loss: 1.1814 - val_accuracy: 0.5710
Epoch 5/12
383/383 [==============================] - 75s 196ms/step - loss: 1.2761 - accuracy: 0.5437 - val_loss: 1.1144 - val_accuracy: 0.5940
Epoch 6/12
383/383 [==============================] - 80s 208ms/step - loss: 1.2134 - accuracy: 0.5687 - val_loss: 1.0581 - val_accuracy: 0.6320
Epoch 7/12
383/

In [6]:
model.fit(x_train, y_train, batch_size=128, epochs=12, verbose=1, validation_data=(x_val, y_val))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
383/383 [==============================] - 76s 199ms/step - loss: 0.9318 - accuracy: 0.6697 - val_loss: 0.8889 - val_accuracy: 0.6820
Epoch 2/12
383/383 [==============================] - 79s 206ms/step - loss: 0.9012 - accuracy: 0.6829 - val_loss: 0.8681 - val_accuracy: 0.6790
Epoch 3/12
383/383 [==============================] - 106s 278ms/step - loss: 0.8722 - accuracy: 0.6912 - val_loss: 0.8832 - val_accuracy: 0.6850
Epoch 4/12
383/383 [==============================] - 73s 191ms/step - loss: 0.8479 - accuracy: 0.6989 - val_loss: 0.8617 - val_accuracy: 0.7000
Epoch 5/12
383/383 [==============================] - 75s 197ms/step - loss: 0.8217 - accuracy: 0.7084 - val_loss: 0.8697 - val_accuracy: 0.6870
Epoch 6/12
383/383 [==============================] - 76s 198ms/step - loss: 0.8077 - accuracy: 0.7120 - val_loss: 0.8398 - val_accuracy: 0.6900
Epoch 7/12
383/383 [==============================] - 76s 197ms/step - loss: 0.7804 - accuracy: 0.7235 - val_loss: 0.8400 - val_a